In [1]:
import requests
# from telethon import TelegramClient
import pandas as pd
import re
# Load dataset and prepare for labeling
df = pd.read_csv('preprocessed_telegram_data.csv')

# Function to tokenize and label
def label_entities(message):
    # Manually labeled entities (example for the task)
    tokens = message.split()
    labels = []
    for token in tokens:
        if token in ['product_example', 'location_example']:
            labels.append('B-PRODUCT' if token == 'product_example' else 'B-LOC')
        else:
            labels.append('O')
    return tokens, labels

# CoNLL format generation
with open('amharic_ner_conll.txt', 'w') as f:
    for _, row in df.iterrows():
        tokens, labels = label_entities(row['clean_message'])
        for token, label in zip(tokens, labels):
            f.write(f"{token}\t{label}\n")
        f.write("\n")  # Separate sentences


In [2]:
import pandas as pd

# Load preprocessed dataset
df = pd.read_csv('preprocessed_telegram_data.csv')

# Define lists of sample products, locations, and price indicators
# These lists should be manually populated based on the dataset
product_list = ['ልጅ ማታለል', 'ካራቦ', 'የቤት ዕቃዎች']  # Example Amharic product names
location_list = ['አዲስ አበባ', 'ቦሌ', 'ባህር ዳር']  # Example Amharic locations
price_list = ['ብር', 'ዋጋ']  # Example price indicators

# Function to tokenize and label entities
def label_entities(message):
    tokens = message.split()  # Tokenize the message by spaces
    labels = []
    
    # Iterate over tokens and label each token
    for token in tokens:
        if token in product_list:
            labels.append('B-PRODUCT')
        elif any(price in token for price in price_list):
            labels.append('B-PRICE')
        elif token in location_list:
            labels.append('B-LOC')
        else:
            labels.append('O')  # Label as 'O' for tokens that do not belong to any entity

    return tokens, labels

# CoNLL format generation
output_file = 'amharic_ner_conll.txt'
with open(output_file, 'w', encoding='utf-8') as f:
    for _, row in df.iterrows():
        tokens, labels = label_entities(row['clean_message'])
        for token, label in zip(tokens, labels):
            f.write(f"{token}\t{label}\n")
        f.write("\n")  # Blank line to separate sentences/messages

print(f"Labeled dataset saved in CoNLL format to {output_file}")


Labeled dataset saved in CoNLL format to amharic_ner_conll.txt


In [4]:
import pandas as pd
import re

# Load the dataset
df = pd.read_csv('preprocessed_telegram_data.csv')

# Example lists of known product names, locations, and price indicators
product_list = ['ልጅ ማታለል', 'ካራቦ', 'የቤት ዕቃዎች']  # Expand this list based on actual data
location_list = ['አዲስ አበባ', 'ቦሌ', 'ባህር ዳር']  # Expand this list based on actual data
price_keywords = ['ብር', 'ዋጋ']  # Words indicating price
price_pattern = r'\d+[,.]?\d*\s?ብር'  # Pattern to match prices (e.g., "1000 ብር")

# Function to label tokens in a message
def label_entities(message):
    tokens = message.split()
    labels = []
    
    # Iterate over tokens
    for i, token in enumerate(tokens):
        if token in product_list:
            labels.append('B-PRODUCT')
        elif re.match(price_pattern, token) or any(price in token for price in price_keywords):
            labels.append('B-PRICE')
        elif token in location_list:
            labels.append('B-LOC')
        else:
            labels.append('O')  # Outside any entity
        
        # Handle multi-word entities (example)
        if i > 0 and labels[-1] == 'B-PRODUCT' and tokens[i-1] in product_list:
            labels[-1] = 'I-PRODUCT'  # Label as inside a product entity
        elif i > 0 and labels[-1] == 'B-LOC' and tokens[i-1] in location_list:
            labels[-1] = 'I-LOC'  # Label as inside a location entity
        elif i > 0 and labels[-1] == 'B-PRICE' and re.match(price_pattern, tokens[i-1]):
            labels[-1] = 'I-PRICE'  # Label as inside a price entity

    return tokens, labels

# CoNLL format generation
output_file = 'amharic_ner_conll.txt'
with open(output_file, 'w', encoding='utf-8') as f:
    for _, row in df.iterrows():
        tokens, labels = label_entities(row['clean_message'])
        for token, label in zip(tokens, labels):
            f.write(f"{token}\t{label}\n")
        f.write("\n")  # Blank line to separate each message

print(f"Labeled dataset saved in CoNLL format to {output_file}")


Labeled dataset saved in CoNLL format to amharic_ner_conll.txt
